# Лабораторная работа №1
---
Требования:
* Python >= 3.X
* NLTK >= 3.2.5

Лабораторную работу необходимо выполнять в данном шаблоне. Результат работы выслать письмом на litvinov.vg@ssau.ru. В теме письма указывать ФИО полностью.

Для выполнения задания необходимо подобрать корпус текстов (художественных) на английском языке. Объем корпуса должен составлять не менее $3 \cdot 10^7$ символов. Далее все действия будут выполнятся над выбранными данными. Для отладки алгоритмов лучше использовать не весь корпус, а лишь его часть.

In [1]:
import os
import random
import requests
import re
import csv
import pandas as pd
from functools import reduce
from collections import Counter 
import nltk
from nltk import RegexpTokenizer, ngrams
from nltk import word_tokenize, wordpunct_tokenize
from tqdm.auto import tqdm

---
### Использованные книги:
---
1. [GIVE THE BOYS A GREAT BIG HAND](https://www.rulit.me/books/give-the-boys-a-great-big-hand-download-679500.html)
2. [Treasure Island by Robert Louis Stevenson](https://www.gutenberg.org/ebooks/120) 
3. [Great Expectations by Charles Dickens](https://www.gutenberg.org/ebooks/1400) 
4. [Hamlet, Prince of Denmark by William Shakespeare](https://www.gutenberg.org/ebooks/1524) 
5. [TUBE TO NOWHERE](https://www.rulit.me/books/tube-to-nowhere-download-679490.html)
6. [YOU HAVE YOURSELF A DEAL](https://www.rulit.me/books/you-have-yourself-a-deal-download-679488.html)
7. [THE WAY THE COOKIE CRUMBLES](https://www.rulit.me/books/the-way-the-cookie-crumbles-download-679487.html) 
8. [YOU MUST BE KIDDING](https://www.rulit.me/books/you-must-be-kidding-download-679486.html) 
9. [The Odyssey by Homer](https://www.gutenberg.org/ebooks/1727) 
10. [DOOM ON THE HILL](https://www.rulit.me/books/doom-on-the-hill-download-679371.html)
11. [Simple Sabotage Field Manual by United States. Office of Strategic Services](https://www.gutenberg.org/ebooks/26184)
12. [THE CHINESE DISKS](https://www.rulit.me/books/the-chinese-disks-download-679286.html) 
13. [The Mysterious Affair at Styles by Agatha Christie](https://www.gutenberg.org/ebooks/863) 
14. [Beowulf: An Anglo-Saxon Epic Poem by J. Lesslie Hall](https://www.gutenberg.org/ebooks/16328) 
15. [NEVER FALL FOR YOUR FIANCEE](https://www.rulit.me/books/never-fall-for-your-fiancee-download-678785.html)
16. [Anna Karenina by graf Leo Tolstoy](https://www.gutenberg.org/ebooks/1399)
17. [On Liberty by John Stuart Mill](https://www.gutenberg.org/ebooks/34901) 
18. [THE GARAUCAN SWINDLE](https://www.rulit.me/books/the-garaucan-swindle-download-678697.html) 
19. [SPOILS OF THE SHADOW](https://www.rulit.me/books/spoils-of-the-shadow-download-678653.html) 
20. [Old Granny Fox by Thornton W. Burgess](https://www.gutenberg.org/ebooks/4980)

In [2]:
with open(r'data/books.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    
len(text)

11506158

In [3]:
text = re.sub(" +"," ",text.replace("\r","").replace("\n"," "))
sample = text[:int(1e5)]
print(text[:1000])

 Ed McBain Give the Boys a Great Big Hand This is for Phyllis and Rick 1 It was raining. It had been raining for three days now, an ugly March rain that washed the brilliance of near-spring with a monochromatic, unrelenting gray. The television forecasters had correctly predicted rain for today and estimated that it would rain tomorrow also. Beyond that, they would not venture an opinion. But it seemed to Patrolman Richard Genero that it had been raining forever, and that it would continue to rain forever, and that eventually he would be washed away into the gutters and then carried into the sewers of Isola and dumped unceremoniously with the other garbage into either the River Harb or the River Dix. North or south, it didn’t make a damn bit of difference: both rivers were polluted; both stank of human waste. Like a man up to his ankles in water in a rapidly sinking rowboat, Genero stood on the corner and surveyed the near-empty streets. His rubber rain cape was as black and as shining

---
#### Шаг №1
Заменить все числа, которые представлены цифрами, их текстовым представлением (т.е. прописью). (1 = one, 23 = twenty three, 1042 = one thousand forty two, и т.п.). Методы библиотек не использовать, алгоритм необходимо написать самостоятельно.

In [4]:
encoder = {
    0:"zero",
    1:"one",
    2:"two",
    3:"three",
    4:"four",
    5:"five",
    6:"six",
    7:"seven",
    8:"eight",
    9:"nine",
    10:"ten",
    11:"eleven",
    12:"twelve",
    13:"thirteen",
    14:"fourteen",
    15:"fifteen",
    16:"sixteen",
    17:"seventeen",
    18:"eigthteen",
    19:"nineteen",
    20:"twenty",
    30:"thirty",
    40:"fourty",
    50:"fifty",
    60:"sixty",
    70:"seventy",
    80:"eighty",
    90:"ninety"
}
def converter(num:str)->str:
    """
    Args:
        num(str/int): input numeric
    Return:
        name in english for numeric
    """
    
    try:
        num_int = int(num)
        if num_int <= 20:
            return encoder[num_int]
        elif num_int < 100:
            decil, number = divmod(num_int,10)
            if number:
                return "{} {}".format(encoder[decil*10],encoder[number%10])
            else:
                return "{}".format(encoder[num_int])
        else:
            num = str(num)
            num_lentgh = len(num)
            if num_lentgh == 3:
                if num_int // 100 and num_int % 100:
                    return "{} hundred {}".format(converter(num[0]), converter(num[1:]))
                else:
                    return "{} hundred".format(converter(num[0]))
            if num_lentgh == 4:
                if num_int // 1000 and num_int % 100:
                    return "{} thousand {}".format(converter(num[0]), converter(num[1:]))
                else:
                    return "{} thousand".format(converter(num[0]))   
    except Exception as e:
        return num

In [5]:
sample_nums = random.sample(range(0, 1500), k=15)
dict(zip(sample_nums, list(map(converter, sample_nums))))

{331: 'three hundred thirty one',
 1043: 'one thousand fourty three',
 988: 'nine hundred eighty eight',
 372: 'three hundred seventy two',
 808: 'eight hundred eight',
 347: 'three hundred fourty seven',
 266: 'two hundred sixty six',
 373: 'three hundred seventy three',
 1459: 'one thousand four hundred fifty nine',
 685: 'six hundred eighty five',
 163: 'one hundred sixty three',
 409: 'four hundred nine',
 1044: 'one thousand fourty four',
 1383: 'one thousand three hundred eighty three',
 883: 'eight hundred eighty three'}

---
#### Шаг №2

Преобразовать текст к нижнему или верхнему регистру.

In [6]:
text = text.lower()
sample = sample.lower()

---
#### Шаг №3
Произвести токенезацию (токенами выступают слова), не учитывая знаки препинания (т.е. необходимо разбить текст на токены (лексемы)):
* С помощью nltk.tokenize.RegexpTokenizer (тут необходимо вспомнить регулярные выражения [posix](https://ru.wikibooks.org/wiki/%D0%A0%D0%B5%D0%B3%D1%83%D0%BB%D1%8F%D1%80%D0%BD%D1%8B%D0%B5_%D0%B2%D1%8B%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D1%8F)).

In [7]:
tokenizer_regexp = RegexpTokenizer(pattern="[^\W\d_]+")
tokens = tokenizer_regexp.tokenize(sample)
tokens = [word for word in tokens if word.isalpha()]
print(tokens[0:100])

['ed', 'mcbain', 'give', 'the', 'boys', 'a', 'great', 'big', 'hand', 'this', 'is', 'for', 'phyllis', 'and', 'rick', 'it', 'was', 'raining', 'it', 'had', 'been', 'raining', 'for', 'three', 'days', 'now', 'an', 'ugly', 'march', 'rain', 'that', 'washed', 'the', 'brilliance', 'of', 'near', 'spring', 'with', 'a', 'monochromatic', 'unrelenting', 'gray', 'the', 'television', 'forecasters', 'had', 'correctly', 'predicted', 'rain', 'for', 'today', 'and', 'estimated', 'that', 'it', 'would', 'rain', 'tomorrow', 'also', 'beyond', 'that', 'they', 'would', 'not', 'venture', 'an', 'opinion', 'but', 'it', 'seemed', 'to', 'patrolman', 'richard', 'genero', 'that', 'it', 'had', 'been', 'raining', 'forever', 'and', 'that', 'it', 'would', 'continue', 'to', 'rain', 'forever', 'and', 'that', 'eventually', 'he', 'would', 'be', 'washed', 'away', 'into', 'the', 'gutters', 'and']


* С помощью nltk.word_tokenize.

In [8]:
tokens = word_tokenize(sample)
tokens = [word for word in tokens if word.isalpha()]
print(tokens[:100])

['ed', 'mcbain', 'give', 'the', 'boys', 'a', 'great', 'big', 'hand', 'this', 'is', 'for', 'phyllis', 'and', 'rick', 'it', 'was', 'raining', 'it', 'had', 'been', 'raining', 'for', 'three', 'days', 'now', 'an', 'ugly', 'march', 'rain', 'that', 'washed', 'the', 'brilliance', 'of', 'with', 'a', 'monochromatic', 'unrelenting', 'gray', 'the', 'television', 'forecasters', 'had', 'correctly', 'predicted', 'rain', 'for', 'today', 'and', 'estimated', 'that', 'it', 'would', 'rain', 'tomorrow', 'also', 'beyond', 'that', 'they', 'would', 'not', 'venture', 'an', 'opinion', 'but', 'it', 'seemed', 'to', 'patrolman', 'richard', 'genero', 'that', 'it', 'had', 'been', 'raining', 'forever', 'and', 'that', 'it', 'would', 'continue', 'to', 'rain', 'forever', 'and', 'that', 'eventually', 'he', 'would', 'be', 'washed', 'away', 'into', 'the', 'gutters', 'and', 'then', 'carried']


#### Шаг №4
Произвести частотный анализ слов. Методы библиотек не использовать, т.е. алгоритм необходимо написать самостоятельно. Cохранить результат в выходной csv файл в порядке убывания частот. Формат выходного файла:

| Word | Count |
| --- | --- |
| cat | 1000 |
| dog | 300 |
| .... | .... |
| butterfly | 34 |

In [9]:
def tokenFreq(tokens:str)->dict:
    res = {}
    for token in tokens:
        if token in res.keys():
            res[token] += 1
        else:
            res[token] = 1
    return res

def sortFreqDict(freqdict:dict)->list:
    res = [(freqdict[key], key) for key in freqdict]
    res.sort()
    res.reverse()
    return res

In [10]:
result = sortFreqDict(tokenFreq(tokens))
print(result[:20])

[(1005, 'the'), (561, 'a'), (451, 'and'), (374, 'you'), (365, 'he'), (358, 'to'), (354, 'i'), (352, 'of'), (262, 'that'), (259, 'was'), (241, 'it'), (240, 'in'), (223, 'said'), (221, 's'), (143, 't'), (139, 'on'), (129, 'his'), (123, 'she'), (118, 'what'), (118, 'for')]


In [11]:
with open("lab1_step4.csv", "a", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(("Word","Count"))
    for tup in result:
        writer.writerow(tup[::-1])

---
#### Шаг №5
Произвести частотный анализ [N-грам](https://en.wikipedia.org/wiki/N-gram) и сохранить в выходной файл в порядке убывания частот (формат файла аналогичен предыдущему примеру, за исключением столбца Word, который заменяется на N-gram. В этой задаче можно использовать готовые библиотеки (смотрим состав библиотеки NLTK). Формат выходного файла:

| N-gram | Count |
| --- | --- |
| cat is | 1000 |
| is a | 300 |
| .... | .... |
| the fish | 34 |

* Биграм

In [12]:
bigramm = Counter(ngrams(tokens, 2)).most_common()
print(bigramm[:100])

[(('of', 'the'), 98), (('in', 'the'), 83), (('on', 'the'), 58), (('to', 'the'), 57), (('carella', 'said'), 49), (('that', 's'), 47), (('do', 'you'), 42), (('don', 't'), 40), (('was', 'a'), 38), (('the', 'bag'), 38), (('it', 'was'), 37), (('i', 'm'), 36), (('he', 'was'), 36), (('and', 'then'), 34), (('of', 'a'), 34), (('and', 'he'), 32), (('in', 'a'), 31), (('at', 'the'), 31), (('it', 's'), 30), (('from', 'the'), 29), (('didn', 't'), 26), (('you', 'know'), 26), (('she', 'said'), 26), (('i', 'don'), 26), (('into', 'the'), 25), (('i', 'mean'), 24), (('and', 'the'), 24), (('with', 'a'), 23), (('there', 'was'), 23), (('he', 'had'), 23), (('out', 'of'), 22), (('he', 'said'), 22), (('the', 'hand'), 22), (('a', 'man'), 21), (('is', 'that'), 21), (('for', 'a'), 20), (('kling', 'said'), 20), (('you', 're'), 19), (('he', 's'), 19), (('said', 'you'), 19), (('had', 'been'), 18), (('a', 'little'), 18), (('you', 'mean'), 18), (('she', 'was'), 18), (('if', 'you'), 18), (('for', 'the'), 17), (('and', '

In [13]:
counter = 0
with open("lab1_step5_bigram.csv", "a", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(("N-gram","Count"))
    for tup in bigramm:
        counter += 1
        row = " ".join(tup[0]), tup[1]
        if counter <= 40:
            print(row)
        writer.writerow(row)

('of the', 98)
('in the', 83)
('on the', 58)
('to the', 57)
('carella said', 49)
('that s', 47)
('do you', 42)
('don t', 40)
('was a', 38)
('the bag', 38)
('it was', 37)
('i m', 36)
('he was', 36)
('and then', 34)
('of a', 34)
('and he', 32)
('in a', 31)
('at the', 31)
('it s', 30)
('from the', 29)
('didn t', 26)
('you know', 26)
('she said', 26)
('i don', 26)
('into the', 25)
('i mean', 24)
('and the', 24)
('with a', 23)
('there was', 23)
('he had', 23)
('out of', 22)
('he said', 22)
('the hand', 22)
('a man', 21)
('is that', 21)
('for a', 20)
('kling said', 20)
('you re', 19)
('he s', 19)
('said you', 19)


* 3-грам

In [14]:
trigramm = Counter(ngrams(tokens, 3)).most_common()
print(trigramm[:100])

[(('i', 'don', 't'), 26), (('of', 'the', 'bag'), 12), (('do', 'you', 'mean'), 11), (('don', 't', 'know'), 11), (('out', 'of', 'the'), 9), (('he', 'did', 'not'), 9), (('do', 'you', 'think'), 8), (('a', 'meat', 'cleaver'), 8), (('i', 'wish', 'i'), 7), (('what', 'the', 'hell'), 7), (('if', 'you', 'are'), 7), (('i', 'm', 'going'), 6), (('you', 're', 'a'), 6), (('that', 's', 'what'), 6), (('do', 'you', 'know'), 6), (('the', 'middle', 'of'), 6), (('there', 'is', 'nothing'), 6), (('the', 'person', 'who'), 6), (('is', 'that', 'right'), 6), (('is', 'that', 'what'), 6), (('there', 'was', 'a'), 6), (('he', 'was', 'a'), 6), (('the', 'missing', 'persons'), 6), (('dozen', 'red', 'roses'), 6), (('m', 'going', 'to'), 5), (('t', 'have', 'to'), 5), (('what', 'do', 'you'), 5), (('a', 'long', 'time'), 5), (('it', 'ain', 't'), 5), (('said', 'and', 'he'), 5), (('shaking', 'his', 'head'), 5), (('what', 'kind', 'of'), 5), (('said', 'you', 're'), 5), (('he', 'didn', 't'), 5), (('what', 'i', 'mean'), 5), (('one

In [15]:
counter = 0
with open("lab1_step5_threegram.csv", "a", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(("N-gram","Count"))
    for tup in trigramm:
        counter += 1
        row = " ".join(tup[0]), tup[1]
        if counter <= 40:
            print(row)
        writer.writerow(row)

('i don t', 26)
('of the bag', 12)
('do you mean', 11)
('don t know', 11)
('out of the', 9)
('he did not', 9)
('do you think', 8)
('a meat cleaver', 8)
('i wish i', 7)
('what the hell', 7)
('if you are', 7)
('i m going', 6)
('you re a', 6)
('that s what', 6)
('do you know', 6)
('the middle of', 6)
('there is nothing', 6)
('the person who', 6)
('is that right', 6)
('is that what', 6)
('there was a', 6)
('he was a', 6)
('the missing persons', 6)
('dozen red roses', 6)
('m going to', 5)
('t have to', 5)
('what do you', 5)
('a long time', 5)
('it ain t', 5)
('said and he', 5)
('shaking his head', 5)
('what kind of', 5)
('said you re', 5)
('he didn t', 5)
('what i mean', 5)
('one of those', 5)
('in the middle', 5)
('you are a', 5)
('are a cop', 5)
('a cop you', 5)


---
#### Шаг №6 (аналогично шагу 3)
Произвести токенезацию, но уже учитывая знаки препинания:
* С помощью nltk.tokenize.RegexpTokenizer.

In [16]:
tokenizer_regexp = RegexpTokenizer(pattern="[^\W\d_]+|\.|,|!|\?")
tokens = tokenizer_regexp.tokenize(sample)
print(tokens[:100])

['ed', 'mcbain', 'give', 'the', 'boys', 'a', 'great', 'big', 'hand', 'this', 'is', 'for', 'phyllis', 'and', 'rick', 'it', 'was', 'raining', '.', 'it', 'had', 'been', 'raining', 'for', 'three', 'days', 'now', ',', 'an', 'ugly', 'march', 'rain', 'that', 'washed', 'the', 'brilliance', 'of', 'near', 'spring', 'with', 'a', 'monochromatic', ',', 'unrelenting', 'gray', '.', 'the', 'television', 'forecasters', 'had', 'correctly', 'predicted', 'rain', 'for', 'today', 'and', 'estimated', 'that', 'it', 'would', 'rain', 'tomorrow', 'also', '.', 'beyond', 'that', ',', 'they', 'would', 'not', 'venture', 'an', 'opinion', '.', 'but', 'it', 'seemed', 'to', 'patrolman', 'richard', 'genero', 'that', 'it', 'had', 'been', 'raining', 'forever', ',', 'and', 'that', 'it', 'would', 'continue', 'to', 'rain', 'forever', ',', 'and', 'that', 'eventually']


* С помощью nltk.word_tokenize.

In [17]:
print(wordpunct_tokenize(sample)[:100])

['ed', 'mcbain', 'give', 'the', 'boys', 'a', 'great', 'big', 'hand', 'this', 'is', 'for', 'phyllis', 'and', 'rick', '1', 'it', 'was', 'raining', '.', 'it', 'had', 'been', 'raining', 'for', 'three', 'days', 'now', ',', 'an', 'ugly', 'march', 'rain', 'that', 'washed', 'the', 'brilliance', 'of', 'near', '-', 'spring', 'with', 'a', 'monochromatic', ',', 'unrelenting', 'gray', '.', 'the', 'television', 'forecasters', 'had', 'correctly', 'predicted', 'rain', 'for', 'today', 'and', 'estimated', 'that', 'it', 'would', 'rain', 'tomorrow', 'also', '.', 'beyond', 'that', ',', 'they', 'would', 'not', 'venture', 'an', 'opinion', '.', 'but', 'it', 'seemed', 'to', 'patrolman', 'richard', 'genero', 'that', 'it', 'had', 'been', 'raining', 'forever', ',', 'and', 'that', 'it', 'would', 'continue', 'to', 'rain', 'forever', ',', 'and']


---
#### Шаг №7 (аналогично шагу 4)
Произвести частотный анализ слов.

In [18]:
def tokenFreq(tokens:str)->dict:
    res = {}
    for token in tokens:
        if token in res.keys():
            res[token] += 1
        else:
            res[token] = 1
    return res

def sortFreqDict(freqdict:dict)->list:
    res = [(freqdict[key], key) for key in freqdict]
    res.sort()
    res.reverse()
    return res

In [19]:
result = sortFreqDict(tokenFreq(tokens))
print(result[:20])

[(1640, '.'), (1462, ','), (1005, 'the'), (568, 'a'), (453, 'and'), (445, '?'), (382, 'you'), (366, 'he'), (359, 'to'), (355, 'i'), (353, 'of'), (266, 'that'), (261, 'was'), (245, 'it'), (242, 'in'), (224, 'said'), (223, 's'), (144, 't'), (139, 'on'), (129, 'his')]


In [20]:
with open("lab1_step7.csv", "a", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(("Word","Count"))
    for tup in result:
        writer.writerow(tup[::-1])

---
#### Шаг №8 (аналогично шагу 5)
Произвести частотный анализ N-грам.

* Биграм

In [21]:
bigramm = Counter(ngrams(tokens, 2)).most_common()
print(bigramm[:100])

[((',', 'and'), 167), (('said', '.'), 135), (('.', 'i'), 119), (('.', 'the'), 100), (('of', 'the'), 98), (('.', 'he'), 97), (('in', 'the'), 83), ((',', 'he'), 72), (('.', 'you'), 66), (('.', 'and'), 60), (('on', 'the'), 58), ((',', 'i'), 58), (('to', 'the'), 57), (('.', '.'), 55), (('well', ','), 52), (('said', ','), 50), (('?', 'i'), 50), ((',', 'carella'), 50), (('carella', 'said'), 50), ((',', 'the'), 49), (('mrs', '.'), 48), (('that', 's'), 47), (('.', 'it'), 46), (('.', 'she'), 46), ((',', 'you'), 45), (('?', 'yes'), 44), (('yes', ','), 43), (('do', 'you'), 42), (('don', 't'), 40), (('.', 'well'), 39), (('the', 'bag'), 39), (('no', ','), 39), (('was', 'a'), 38), (('.', 'what'), 38), (('it', 'was'), 37), ((',', 'but'), 37), (('he', 'was'), 37), ((',', 'mrs'), 37), (('and', 'then'), 36), (('i', 'm'), 36), (('of', 'a'), 35), ((',', 'that'), 34), (('.', 'but'), 32), (('and', 'he'), 32), (('.', 'we'), 32), ((',', 'sir'), 32), (('?', 'no'), 32), (('in', 'a'), 31), (('.', 'a'), 31), (('y

In [22]:
counter = 0
with open("lab1_step8_bigram.csv", "a", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(("N-gram","Count"))
    for tup in bigramm:
        counter += 1
        row = " ".join(tup[0]), tup[1]
        if counter <= 40:
            print(row)
        writer.writerow(row)

(', and', 167)
('said .', 135)
('. i', 119)
('. the', 100)
('of the', 98)
('. he', 97)
('in the', 83)
(', he', 72)
('. you', 66)
('. and', 60)
('on the', 58)
(', i', 58)
('to the', 57)
('. .', 55)
('well ,', 52)
('said ,', 50)
('? i', 50)
(', carella', 50)
('carella said', 50)
(', the', 49)
('mrs .', 48)
('that s', 47)
('. it', 46)
('. she', 46)
(', you', 45)
('? yes', 44)
('yes ,', 43)
('do you', 42)
('don t', 40)
('. well', 39)
('the bag', 39)
('no ,', 39)
('was a', 38)
('. what', 38)
('it was', 37)
(', but', 37)
('he was', 37)
(', mrs', 37)
('and then', 36)
('i m', 36)


* 3-грам

In [23]:
trigramm = Counter(ngrams(tokens, 3)).most_common()
print(trigramm[:100])

[((',', 'carella', 'said'), 43), (('carella', 'said', '.'), 41), (('.', 'well', ','), 37), ((',', 'mrs', '.'), 37), ((',', 'and', 'he'), 28), (('.', '.', '.'), 28), (('mrs', '.', 'androvich'), 28), (('i', 'don', 't'), 26), ((',', 'sir', '.'), 24), (('?', 'yes', '.'), 23), (('?', 'no', ','), 21), ((',', 'that', 's'), 20), (('?', 'yes', ','), 20), (('.', 'that', 's'), 19), ((',', 'kling', 'said'), 19), ((',', 'she', 'said'), 18), (('mrs', '.', 'livingston'), 18), (('.', 'i', 'm'), 17), ((',', 'and', 'then'), 17), ((',', 'he', 'said'), 17), (('.', 'do', 'you'), 17), (('i', 'mean', ','), 16), (('.', 'yes', ','), 16), (('.', 'it', 'was'), 14), (('.', 'it', 's'), 14), (('he', 'said', '.'), 14), ((',', 'you', 'know'), 14), (('?', 'i', 'don'), 14), (('don', 't', 'know'), 14), (('.', 'he', 'was'), 13), (('said', '.', 'you'), 13), ((',', 'meyer', 'said'), 13), (('kling', 'said', '.'), 13), (('she', 'said', '.'), 13), (('.', 'i', 'mean'), 12), (('of', 'the', 'bag'), 12), (('.', 'oh', ','), 12), (

In [24]:
counter = 0
with open("lab1_step8_threegram.csv", "a", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(("N-gram","Count"))
    for tup in trigramm:
        counter += 1
        row = " ".join(tup[0]), tup[1]
        if counter <= 40:
            print(row)
        writer.writerow(row)

(', carella said', 43)
('carella said .', 41)
('. well ,', 37)
(', mrs .', 37)
(', and he', 28)
('. . .', 28)
('mrs . androvich', 28)
('i don t', 26)
(', sir .', 24)
('? yes .', 23)
('? no ,', 21)
(', that s', 20)
('? yes ,', 20)
('. that s', 19)
(', kling said', 19)
(', she said', 18)
('mrs . livingston', 18)
('. i m', 17)
(', and then', 17)
(', he said', 17)
('. do you', 17)
('i mean ,', 16)
('. yes ,', 16)
('. it was', 14)
('. it s', 14)
('he said .', 14)
(', you know', 14)
('? i don', 14)
('don t know', 14)
('. he was', 13)
('said . you', 13)
(', meyer said', 13)
('kling said .', 13)
('she said .', 13)
('. i mean', 12)
('of the bag', 12)
('. oh ,', 12)
('. yeah ,', 12)
('? well ,', 12)
('. androvich ?', 12)
